## Tips
    - 지금까지 강의를 들으면서 이해가 어려웠던 부분들을 설명한다.
1. Reminder: 최대 가능도 추정(Maximum Likelihood Estimation)
2. Reminder: Optimization via Gradient Descent
3. Reminder: 과적합(Overfitting)과 정규화(Regurlarization)
4. 훈련 세트와 테스트 세트
5. 학습률(Learning Rate)
6. 데이터 전처리(Data Preprocessing)

### Maximum Likelihood Estimation
- 압정이 어떻게 떨어지는지 종류는 2가지. class1, class2
- 머신러닝을 통해 둘 중에 어떻게 떨어질지 예측하자.
- 둘 중에 예측이므로 베르누이 분포이다.
- 베르누이 분포를 여러 번 하면 이항분포가 된다. 
- 100번 시행 중에 class1이 27번 나왔다고 치자. n=100, k=27
- 이항분포 식에 n=100, k=27을 대입한다. 
- 세타에 관련된 식이 나온다. 이 식에서 likelihood (가능성)이 가장 큰 세타값을 구한다.

### Optimization via Gradient Descent
- 우리가 구한 observation을 가장 잘 설명하는 확률분포의 파라미터를 찾는 과정.
- 어떻게 구하느냐? 그래프의 기울기를 이용해서 구함.
- 최대를 찾으므로 Gradient ascent.
- 최소를 찾을 때는 Gradient descent.

### Overfitting
- 주어진 데이터에 대해서 과도하게 확률밀도함수가 fitting되어버린 것을 overfitting이라고 한다.(너무 과도하게 잘 설명하는 것을 막고자 함)
- overfitting을 최소화 해야 한다.
- overfitting 최소화하는 법
    - training set, dev set=validation set, test set 으로 나눈다.
    - training set에서 overfitting이 되었는지 test set으로 확인할 수 있다.
    - development set은 test set에 대해서도 overfitting이 되는 것을 방지하는 역할을 한다.(여러 번 fitting하다보면 이런 일이 일어날 수 있다.)

    - training set 학습, dev set 검증, test set 으로 test.
    - epoch가 커질수록 train loss는 줄어들고 dev loss는 줄어들다가 다시 커진다.
    - train loss도 작고 dev loss도 작은 부분의 epoch에서 모델을 선택하면 된다.
    - 많은 데이터를 모은다.
    - feature를 적게 사용한다.
        - feature란 어떤 분포를 잘 설명하는 특징을 말한다.
    - Regularization
        - Early stopping
            - validation loss가 더이상 낮아지지 않을 때 멈춘다.
        - Reducing Network Size
            - Neural network가 학습할 수 있는 양을 줄인다.
        - Weight Decay
            - Neural network weight 파라미터의 크기를 제한한다.
        - Dropout
        - Batch Normalization


### Basic Approach to Train DNN
1. Neural network architecture를 설계한다.
2. Train and check that model is over-fitted
    - If it is not, increase the model size(deeper and wider)
    - If it is, add regularization, such as drop=out, batch-normalization
3. Repeat from step 2
----
이제 이론 끝! 실습을 진행하자.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

In [4]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear=nn.Linear(3,3)
    def forward(self,x):
        return self.linear(x)

In [5]:
model=SoftmaxClassifierModel()

In [6]:
optimizer=optim.SGD(model.parameters(),lr=0.1)

In [7]:
def train(model, optimizer, x_train, y_train):
    nb_epochs=20
    for epoch in range(nb_epochs):
        prediction=model(x_train)
        cost=F.cross_entropy(prediction,y_train)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))
        

In [11]:
def test(model,optimizer,x_test,y_test):
    prediction=model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count=(predicted_classes==y_test).sum().item()
    cost=F.cross_entropy(prediction,y_test)
    
    print('Accuracy: {}% Cost: {:.6f}'.format(correct_count/len(y_test)*100,cost.item()))

In [12]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 0.983424
Epoch    1/20 Cost: 0.977591
Epoch    2/20 Cost: 0.971864
Epoch    3/20 Cost: 0.966240
Epoch    4/20 Cost: 0.960718
Epoch    5/20 Cost: 0.955295
Epoch    6/20 Cost: 0.949968
Epoch    7/20 Cost: 0.944736
Epoch    8/20 Cost: 0.939596
Epoch    9/20 Cost: 0.934546
Epoch   10/20 Cost: 0.929585
Epoch   11/20 Cost: 0.924709
Epoch   12/20 Cost: 0.919918
Epoch   13/20 Cost: 0.915210
Epoch   14/20 Cost: 0.910582
Epoch   15/20 Cost: 0.906033
Epoch   16/20 Cost: 0.901561
Epoch   17/20 Cost: 0.897164
Epoch   18/20 Cost: 0.892841
Epoch   19/20 Cost: 0.888590


In [13]:
test(model, optimizer, x_test, y_test)

Accuracy: 66.66666666666666% Cost: 0.791733


### Learning Rate
- gradient descent에서의 알파 값.
- learning rate이 너무 크면 diverge하면서 cost가 점점 늘어난다. (overshooting)
- 너무 작으면 cost가 거의 줄어들지 않는다.

In [14]:
#learning rate이 너무 클 때
model=SoftmaxClassifierModel()
optimizer=optim.SGD(model.parameters(),lr=1e5)
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 976950.750000
Epoch    2/20 Cost: 1279135.125000
Epoch    3/20 Cost: 1198379.125000
Epoch    4/20 Cost: 1098825.750000
Epoch    5/20 Cost: 1968197.625000
Epoch    6/20 Cost: 284763.218750
Epoch    7/20 Cost: 1532260.000000
Epoch    8/20 Cost: 1651504.125000
Epoch    9/20 Cost: 521878.437500
Epoch   10/20 Cost: 1397263.250000
Epoch   11/20 Cost: 750986.250000
Epoch   12/20 Cost: 918691.562500
Epoch   13/20 Cost: 1487888.250000
Epoch   14/20 Cost: 1582260.125000
Epoch   15/20 Cost: 685818.062500
Epoch   16/20 Cost: 1140048.750000
Epoch   17/20 Cost: 940566.562500
Epoch   18/20 Cost: 931638.250000
Epoch   19/20 Cost: 1971322.625000


In [15]:
#learning rate이 너무 작을 때
model=SoftmaxClassifierModel()
optimizer=optim.SGD(model.parameters(),lr=1e-1)
train(model, optimizer, x_train, y_train)
# 어떤 값으로 정해져있지는 않아서 하면서 조금씩 조절하면 된다.

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 1.334307
Epoch    2/20 Cost: 1.047911
Epoch    3/20 Cost: 0.996043
Epoch    4/20 Cost: 0.985740
Epoch    5/20 Cost: 0.977224
Epoch    6/20 Cost: 0.970065
Epoch    7/20 Cost: 0.963589
Epoch    8/20 Cost: 0.957562
Epoch    9/20 Cost: 0.951825
Epoch   10/20 Cost: 0.946302
Epoch   11/20 Cost: 0.940942
Epoch   12/20 Cost: 0.935719
Epoch   13/20 Cost: 0.930612
Epoch   14/20 Cost: 0.925613
Epoch   15/20 Cost: 0.920711
Epoch   16/20 Cost: 0.915902
Epoch   17/20 Cost: 0.911182
Epoch   18/20 Cost: 0.906547
Epoch   19/20 Cost: 0.901994


### Data Preprocessing
- 데이터 전처리

In [16]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [18]:
# 데이터 전처리 방법 중 standardization 이용. 정규화!

mu=x_train.mean(dim=0)
sigma=x_train.std(dim=0)
norm_x_train=(x_train-mu)/sigma

print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


In [19]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29729.951172
Epoch    1/20 Cost: 18889.082031
Epoch    2/20 Cost: 12048.975586
Epoch    3/20 Cost: 7699.843262
Epoch    4/20 Cost: 4924.700684
Epoch    5/20 Cost: 3151.020264
Epoch    6/20 Cost: 2016.562866
Epoch    7/20 Cost: 1290.709106
Epoch    8/20 Cost: 826.216003
Epoch    9/20 Cost: 528.952271
Epoch   10/20 Cost: 338.703339
Epoch   11/20 Cost: 216.940048
Epoch   12/20 Cost: 139.006989
Epoch   13/20 Cost: 89.125130
Epoch   14/20 Cost: 57.196079
Epoch   15/20 Cost: 36.757317
Epoch   16/20 Cost: 23.672049
Epoch   17/20 Cost: 15.293400
Epoch   18/20 Cost: 9.927166
Epoch   19/20 Cost: 6.488903
